In [1]:
def add(x,y):
    print(vars())

add(1,2)


{'y': 2, 'x': 1}


In [3]:
def outer(z):
    print(locals())
    def inner(y):
        print(locals())
        return x + y + z
    return inner

class A (object):
    def __init__ (self, name):
        self.name = name

x = 100
a0 = A('park')
a0.name = 'park'
a = outer(5) # z
print(a)
print(a(5)) # y

outer.a = a
print(outer.a(10))

{'z': 5}
<function outer.<locals>.inner at 0x0000000004C0CAE8>
{'y': 5, 'z': 5}
110
{'y': 10, 'z': 5}
115


In [4]:
def outer(x):
    def inner(y):
        nonlocal x
        x += y
        return x
    return inner

inner = outer(5)
print(inner(5))

print(inner(10))
print(inner.__closure__[0].cell_contents)

10
20
20


In [5]:
from functools import wraps

def dec(func):
    @wraps(func) # 메타 정보 관리
    def wrapper(*args, **kwargs): # *: 가변인자 / **: 키워드인자
        result = func(*args, **kwargs)
        return result
    return wrapper

# @dec: 함수의 재사용 / 클래스에도 사용됨 / 단점: 메타 정보 날라감
@dec # @dec 사용하지 않을 경우: add = dec(add) ; def add(x, y): return x + y ;
def add(x, y):
    return x + y

@dec
def multiply(x, y):
    return x * y

print(add(5, 5))
print(multiply(4, 3))

10
12


### 데코레이터가 실행되는 방식

    데코레이터는 함수를 함쳐서 내부함수를 실행되는 함수이 이름과 동일한 변수에 할당한다.
    이를 가지고 실행하면 실제 처리가 된다.

In [6]:
add = dec(add)
print(add(10, 10))
# add = dec(1, 2)(add) ?????

20


### 클래스 와 스태틱 메소드에 대한 데코레이터 처리

    클래스 메소드는 클래스이다.
    클래스메소드를 만들 때 클래스를 이용해서 데코레이터를 처리한다.
    

In [36]:
dir(classmethod)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__func__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__isabstractmethod__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [35]:
type(classmethod)

type

### static method는 클래스 내에서 함수를 처리하는 구조를 제공할 뿐이다.


In [7]:
type(staticmethod)

type

In [8]:
class A (object):
    @staticmethod
    def add(x, y):
        return x + y   
    
A.add(5, 4)

9

### 프로퍼티 구조

    프로퍼티로 데코레이터 처리를 하면 함수의 이름이 클래스 내부에 들어간다.
    실제 참조시 클래스부터 처리하게 되어 있다.
    

In [20]:
class Person(object):
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    @property
    def name(self):
        return self._name + '!!!'
    
    @property
    def age(self):
        return 'You\'re ' + self._age + '? OMG!!!'
    
print(Person.__dict__)

{'age': <property object at 0x0000000004EBB868>, '__doc__': None, '__init__': <function Person.__init__ at 0x0000000004EC8EA0>, 'name': <property object at 0x0000000004EBBD18>, '__weakref__': <attribute '__weakref__' of 'Person' objects>, '__dict__': <attribute '__dict__' of 'Person' objects>, '__module__': '__main__'}


### 프로퍼티 처리

    프로퍼티는 메소드를 이용해서 속성을 접근하려는 패턴

In [68]:
class Person(object):
    def __init__(self, name, age):
        self._name = name
        self._age = age
        
    @property
    def name(self):
        return self._name + '!!!'
    
    @property
    def age(self):
        return 'You\'re ' + self._age + '? OMG!!!'

p = Person('Choi', 'Twenty six year-old')
print(p.name)
print(p.age)

Choi!!!
You're Twenty six year-old? OMG!!!


### 프로퍼티 처리시 주의사항

    인스턴스에 동일한 이름이 setting 되지 않도록 처리해야 한다.
    
    동일한 이름으로 인스턴스에 세팅하면 반드시 .setter 처리가 되어야 한다.
    
    

In [19]:
class Person(object):
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    @property
    def name(self):
        return self.name + '!!!'
    
    @property
    def age(self):
        return 'You\'re ' + str(self.age) + '? OMG!!!'

print(Person.__dict__)
c = Person("name",30)


{'age': <property object at 0x0000000004EC9E58>, '__doc__': None, '__init__': <function Person.__init__ at 0x0000000004EC8620>, 'name': <property object at 0x0000000004EC9778>, '__weakref__': <attribute '__weakref__' of 'Person' objects>, '__dict__': <attribute '__dict__' of 'Person' objects>, '__module__': '__main__'}


AttributeError: can't set attribute

###  프로퍼티로 계산된 결과만 처리 시에 주의사항

    관행적으로 세팅을 하게되면 실제 속성이 없으므로 예외가 발생할 수 있다.
    이런 점을 해결하기 위해 setter 연산을 작성해서 처리가 필요할 경우도 발생한다.
    예제는 간단히 변경만 처리했다.
    

In [25]:
class Person(object):
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name + '!!!'
    
    @name.setter
    def name(self, value):
        self._name = value


c = Person("name")

c.age = 100

print(c.age)

100


### 프로퍼티로 세팅처리하기 

In [61]:
class Person(object):
    def __init__ (self, name):
        self._name = name
    
    @property
    def name(self):
        return self._name + '!!!'
    
    @name.setter
    def name(self, value):
        self._name = value
        
p = Person('Park')
print(p.name)
p.name = 'Choi' # set해서, @name.setter 발동!
print(p.name)

Park!!!
Choi!!!
